In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🤖 Building Your First Agent - Practice Notebook\n",
    "\n",
    "This notebook guides you through building a Reflection Agent step-by-step.\n",
    "\n",
    "**What you'll build:** An agent that answers questions and improves its answers through self-reflection.\n",
    "\n",
    "## Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Imports\n",
    "from datetime import datetime\n",
    "from typing import List, Dict, Optional\n",
    "import json\n",
    "\n",
    "# If using OpenAI\n",
    "# !pip install openai\n",
    "# from openai import OpenAI\n",
    "\n",
    "# If using Anthropic\n",
    "# !pip install anthropic\n",
    "# from anthropic import Anthropic\n",
    "\n",
    "print(\"✅ Imports complete!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part 1: Build Memory Systems\n",
    "\n",
    "First, let's create the memory components."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class ShortTermMemory:\n",
    "    \"\"\"Working memory for current task\"\"\"\n",
    "    \n",
    "    def __init__(self, max_size: int = 5):\n",
    "        self.max_size = max_size\n",
    "        self.context = []\n",
    "    \n",
    "    def add(self, item: Dict):\n",
    "        \"\"\"Add item to working memory\"\"\"\n",
    "        self.context.append(item)\n",
    "        \n",
    "        # Keep only recent items\n",
    "        if len(self.context) > self.max_size:\n",
    "            self.context = self.context[-self.max_size:]\n",
    "    \n",
    "    def get(self) -> List[Dict]:\n",
    "        \"\"\"Get current context\"\"\"\n",
    "        return self.context\n",
    "    \n",
    "    def clear(self):\n",
    "        \"\"\"Clear working memory\"\"\"\n",
    "        self.context = []\n",
    "\n",
    "# Test it\n",
    "stm = ShortTermMemory(max_size=3)\n",
    "stm.add({\"step\": 1, \"action\": \"search\"})\n",
    "stm.add({\"step\": 2, \"action\": \"read\"})\n",
    "print(\"Short-term memory:\", stm.get())\n",
    "print(\"✅ Short-term memory works!\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class EpisodicMemory:\n",
    "    \"\"\"Log of all actions and results\"\"\"\n",
    "    \n",
    "    def __init__(self):\n",
    "        self.episodes = []\n",
    "    \n",
    "    def log(self, action_type: str, details: Dict):\n",
    "        \"\"\"Log an action\"\"\"\n",
    "        episode = {\n",
    "            \"timestamp\": datetime.now().isoformat(),\n",
    "            \"action\": action_type,\n",
    "            \"details\": details\n",
    "        }\n",
    "        self.episodes.append(episode)\n",
    "    \n",
    "    def get_history(self) -> List[Dict]:\n",
    "        \"\"\"Get all episodes\"\"\"\n",
    "        return self.episodes\n",
    "    \n",
    "    def get_summary(self) -> str:\n",
    "        \"\"\"Summarize the session\"\"\"\n",
    "        summary = f\"Total actions: {len(self.episodes)}\\n\"\n",
    "        \n",
    "        action_counts = {}\n",
    "        for ep in self.episodes:\n",
    "            action = ep[\"action\"]\n",
    "            action_counts[action] = action_counts.get(action, 0) + 1\n",
    "        \n",
    "        for action, count in action_counts.items():\n",
    "            summary += f\"  - {action}: {count}\\n\"\n",
    "        \n",
    "        return summary\n",
    "\n",
    "# Test it\n",
    "em = EpisodicMemory()\n",
    "em.log(\"perceive\", {\"context\": \"ready\"})\n",
    "em.log(\"plan\", {\"action\": \"search\"})\n",
    "print(\"Episodic memory:\", em.get_summary())\n",
    "print(\"✅ Episodic memory works!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part 2: Create LLM Interface\n",
    "\n",
    "Choose your LLM provider and uncomment the appropriate section:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# OPTION 1: Mock LLM (for testing without API)\n",
    "class SimpleLLM:\n",
    "    def __init__(self):\n",
    "        self.call_count = 0\n",
    "    \n",
    "    def complete(self, prompt: str) -> str:\n",
    "        self.call_count += 1\n",
    "        print(f\"\\n[LLM CALL #{self.call_count}]\")\n",
    "        print(f\"Prompt: {prompt[:100]}...\")\n",
    "        \n",
    "        # Mock response\n",
    "        if \"critique\" in prompt.lower():\n",
    "            return \"Quality: 9/10. The answer is good but could be more concise.\"\n",
    "        else:\n",
    "            return \"This is a mock answer from the LLM. In reality, this would be a real response.\"\n",
    "\n",
    "llm = SimpleLLM()\n",
    "print(\"✅ Mock LLM ready (for testing)\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# OPTION 2: OpenAI (uncomment to use)\n",
    "# import os\n",
    "# from openai import OpenAI\n",
    "\n",
    "# class SimpleLLM:\n",
    "#     def __init__(self, model_name=\"gpt-4o-mini\"):\n",
    "#         self.client = OpenAI(api_key=os.getenv(\"OPENAI_API_KEY\"))\n",
    "#         self.model_name = model_name\n",
    "#         self.call_count = 0\n",
    "    \n",
    "#     def complete(self, prompt: str) -> str:\n",
    "#         self.call_count += 1\n",
    "#         print(f\"\\n[LLM CALL #{self.call_count}]\")\n",
    "        \n",
    "#         response = self.client.chat.completions.create(\n",
    "#             model=self.model_name,\n",
    "#             messages=[\n",
    "#                 {\"role\": \"system\", \"content\": \"You are a helpful assistant.\"},\n",
    "#                 {\"role\": \"user\", \"content\": prompt}\n",
    "#             ],\n",
    "#             temperature=0.7,\n",
    "#             max_tokens=1000\n",
    "#         )\n",
    "#         return response.choices[0].message.content\n",
    "\n",
    "# llm = SimpleLLM()\n",
    "# print(\"✅ OpenAI LLM ready\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part 3: Build the Agent\n",
    "\n",
    "Now let's create the complete Reflection Agent!"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class ReflectionAgent:\n",
    "    \"\"\"Agent that improves answers through self-reflection\"\"\"\n",
    "    \n",
    "    def __init__(self, llm, max_iterations: int = 3):\n",
    "        self.llm = llm\n",
    "        self.max_iterations = max_iterations\n",
    "        \n",
    "        # Memory systems\n",
    "        self.short_term = ShortTermMemory()\n",
    "        self.episodic = EpisodicMemory()\n",
    "        \n",
    "        # Agent state\n",
    "        self.current_question = None\n",
    "        self.current_answer = None\n",
    "        self.iteration = 0\n",
    "        self.satisfied = False\n",
    "    \n",
    "    def run(self, question: str) -> str:\n",
    "        \"\"\"Main agent loop\"\"\"\n",
    "        self.current_question = question\n",
    "        self.iteration = 0\n",
    "        self.satisfied = False\n",
    "        \n",
    "        print(f\"\\n{'='*60}\")\n",
    "        print(f\"QUESTION: {question}\")\n",
    "        print(f\"{'='*60}\")\n",
    "        \n",
    "        while not self.satisfied and self.iteration < self.max_iterations:\n",
    "            self.iteration += 1\n",
    "            print(f\"\\n--- ITERATION {self.iteration} ---\")\n",
    "            \n",
    "            # THE AGENT LOOP\n",
    "            context = self.perceive()\n",
    "            action = self.plan(context)\n",
    "            result = self.act(action)\n",
    "            self.reflect(result)\n",
    "        \n",
    "        print(f\"\\n{'='*60}\")\n",
    "        print(f\"FINAL ANSWER:\\n{self.current_answer}\")\n",
    "        print(f\"{'='*60}\")\n",
    "        \n",
    "        return self.current_answer\n",
    "    \n",
    "    def perceive(self) -> Dict:\n",
    "        \"\"\"Gather current context\"\"\"\n",
    "        context = {\n",
    "            \"question\": self.current_question,\n",
    "            \"current_answer\": self.current_answer,\n",
    "            \"iteration\": self.iteration\n",
    "        }\n",
    "        \n",
    "        self.episodic.log(\"perceive\", {\"iteration\": self.iteration})\n",
    "        print(f\"[PERCEIVE] Gathering context for iteration {self.iteration}\")\n",
    "        \n",
    "        return context\n",
    "    \n",
    "    def plan(self, context: Dict) -> Dict:\n",
    "        \"\"\"Decide what to do next\"\"\"\n",
    "        if context[\"current_answer\"] is None:\n",
    "            action = {\"type\": \"generate_answer\"}\n",
    "        else:\n",
    "            action = {\"type\": \"improve_answer\"}\n",
    "        \n",
    "        self.episodic.log(\"plan\", action)\n",
    "        print(f\"[PLAN] Action: {action['type']}\")\n",
    "        \n",
    "        return action\n",
    "    \n",
    "    def act(self, action: Dict) -> Dict:\n",
    "        \"\"\"Execute the planned action\"\"\"\n",
    "        if action[\"type\"] == \"generate_answer\":\n",
    "            result = self._generate_initial_answer()\n",
    "        else:\n",
    "            result = self._improve_answer()\n",
    "        \n",
    "        self.short_term.add({\"action\": action[\"type\"], \"success\": result[\"success\"]})\n",
    "        self.episodic.log(\"act\", {\"action_type\": action[\"type\"]})\n",
    "        \n",
    "        return result\n",
    "    \n",
    "    def _generate_initial_answer(self) -> Dict:\n",
    "        \"\"\"Generate the first answer\"\"\"\n",
    "        print(f\"[ACT] Generating initial answer...\")\n",
    "        \n",
    "        prompt = f\"Question: {self.current_question}\\n\\nProvide a clear and accurate answer.\"\n",
    "        answer = self.llm.complete(prompt)\n",
    "        self.current_answer = answer\n",
    "        \n",
    "        print(f\"[ACT] Answer generated ({len(answer)} chars)\")\n",
    "        return {\"success\": True, \"answer\": answer}\n",
    "    \n",
    "    def _improve_answer(self) -> Dict:\n",
    "        \"\"\"Improve based on critique\"\"\"\n",
    "        print(f\"[ACT] Improving answer...\")\n",
    "        \n",
    "        recent = self.short_term.get()\n",
    "        last_critique = None\n",
    "        for item in reversed(recent):\n",
    "            if \"critique\" in item:\n",
    "                last_critique = item[\"critique\"]\n",
    "                break\n",
    "        \n",
    "        prompt = f\"\"\"\n",
    "Question: {self.current_question}\n",
    "Current Answer: {self.current_answer}\n",
    "Critique: {last_critique}\n",
    "\n",
    "Improve the answer based on the critique.\n",
    "\"\"\"\n",
    "        \n",
    "        improved = self.llm.complete(prompt)\n",
    "        self.current_answer = improved\n",
    "        \n",
    "        print(f\"[ACT] Answer improved ({len(improved)} chars)\")\n",
    "        return {\"success\": True, \"answer\": improved}\n",
    "    \n",
    "    def reflect(self, result: Dict):\n",
    "        \"\"\"Evaluate and decide next steps\"\"\"\n",
    "        print(f\"[REFLECT] Evaluating quality...\")\n",
    "        \n",
    "        critique_prompt = f\"\"\"\n",
    "Question: {self.current_question}\n",
    "Answer: {self.current_answer}\n",
    "\n",
    "Critically evaluate this answer:\n",
    "1. Is it accurate?\n",
    "2. Is it complete?\n",
    "3. What could be improved?\n",
    "\n",
    "Rate quality 1-10. Format: Quality: [score]/10\n",
    "\"\"\"\n",
    "        \n",
    "        critique = self.llm.complete(critique_prompt)\n",
    "        self.short_term.add({\"critique\": critique})\n",
    "        self.episodic.log(\"reflect\", {\"iteration\": self.iteration})\n",
    "        \n",
    "        # Check if satisfied\n",
    "        if \"9\" in critique or \"10\" in critique:\n",
    "            self.satisfied = True\n",
    "            print(f\"[REFLECT] ✅ Satisfied with quality!\")\n",
    "        elif self.iteration >= self.max_iterations:\n",
    "            print(f\"[REFLECT] ⏱️ Max iterations reached\")\n",
    "        else:\n",
    "            print(f\"[REFLECT] 🔄 Will improve in next iteration\")\n",
    "    \n",
    "    def get_session_summary(self) -> str:\n",
    "        return self.episodic.get_summary()\n",
    "\n",
    "print(\"✅ ReflectionAgent class created!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part 4: Test Your Agent!\n",
    "\n",
    "Now let's run it with different questions:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create agent instance\n",
    "agent = ReflectionAgent(llm, max_iterations=3)\n",
    "\n",
    "# Test question\n",
    "question = \"What is the difference between a language model and an AI agent?\"\n",
    "\n",
    "# Run it!\n",
    "final_answer = agent.run(question)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# View session summary\n",
    "print(\"\\nSESSION SUMMARY:\")\n",
    "print(agent.get_session_summary())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part 5: Experiments\n",
    "\n",
    "Try different questions and observe how the agent improves:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Experiment 1: Simple question\n",
    "agent1 = ReflectionAgent(llm, max_iterations=2)\n",
    "agent1.run(\"What is Python?\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Experiment 2: Complex question\n",
    "agent2 = ReflectionAgent(llm, max_iterations=3)\n",
    "agent2.run(\"Explain how neural networks learn\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Experiment 3: Your own question!\n",
    "agent3 = ReflectionAgent(llm, max_iterations=3)\n",
    "my_question = \"YOUR QUESTION HERE\"\n",
    "agent3.run(my_question)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part 6: Analysis\n",
    "\n",
    "Let's analyze how the agent performed:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get full history\n",
    "history = agent.episodic.get_history()\n",
    "\n",
    "print(\"Full Episode History:\")\n",
    "for i, episode in enumerate(history, 1):\n",
    "    print(f\"\\n{i}. {episode['action']} at {episode['timestamp'][:19]}\")\n",
    "    print(f\"   Details: {episode['details']}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Count LLM calls\n",
    "print(f\"\\nTotal LLM calls made: {llm.call_count}\")\n",
    "print(f\"Cost estimate (if using GPT-4): ~${llm.call_count * 0.01:.2f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎯 Your Challenges\n",
    "\n",
    "Try these modifications:\n",
    "\n",
    "1. **Add a confidence score** - Make the agent report how confident it is\n",
    "2. **Save to file** - Export the final answer and history to JSON\n",
    "3. **Compare iterations** - Show the difference between first and final answer\n",
    "4. **Add more reflection criteria** - Check for clarity, citations, examples\n",
    "5. **Multi-turn questions** - Let the agent handle follow-up questions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Challenge 1: Save session to file\n",
    "def save_session(agent, filename=\"agent_session.json\"):\n",
    "    \"\"\"Save agent session to JSON file\"\"\"\n",
    "    session_data = {\n",
    "        \"question\": agent.current_question,\n",
    "        \"final_answer\": agent.current_answer,\n",
    "        \"iterations\": agent.iteration,\n",
    "        \"history\": agent.episodic.get_history()\n",
    "    }\n",
    "    \n",
    "    with open(filename, 'w') as f:\n",
    "        json.dump(session_data, f, indent=2)\n",
    "    \n",
    "    print(f\"✅ Session saved to {filename}\")\n",
    "\n",
    "# Use it\n",
    "save_session(agent, \"my_first_agent_session.json\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎉 Congratulations!\n",
    "\n",
    "You've built your first complete AI agent! \n",
    "\n",
    "**What you accomplished:**\n",
    "- ✅ Implemented the agent loop (Perceive → Plan → Act → Reflect)\n",
    "- ✅ Added memory systems (short-term + episodic)\n",
    "- ✅ Created self-reflection capability\n",
    "- ✅ Built a working autonomous system\n",
    "\n",
    "**Next steps:**\n",
    "- Modify this agent for different tasks\n",
    "- Add tools (web search, calculator, etc.)\n",
    "- Build more complex agents\n",
    "- Explore multi-agent systems"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}